## Setup

Mount Google Drive

In [ ]:
if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  drive.mount('/content/drive')

In [ ]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.data import Dataset 
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Conv2D, Input
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
%matplotlib inline

Paths and variables

In [ ]:
DATA_FILE_PATH = '/content/drive/MyDrive/GOL/data/game_of_life_100-100000.h5'
H5PY_DATASET_NAME = 'gol_frames'
FRAME_SIZE = 100
N_SERIES = 5
EPOCHS = 1

## Loading and preprocessing

noisy_movies = np.zeros((n_samples, n_frames, row, col, 1), dtype=np.int)

In [ ]:
def DataGenerator(batch_size, h5_path, data_share=None):
  with h5py.File(h5_path, 'r') as f:
    if data_share == None:
      dataset = f[H5PY_DATASET_NAME]
    else:
      dataset = f[H5PY_DATASET_NAME][:data_share]
    counter = 0
    while True:
      input_frames = np.zeros((batch_size, N_SERIES, FRAME_SIZE, FRAME_SIZE, 1), dtype=np.int)
      output_frame = np.zeros((batch_size, 1, FRAME_SIZE, FRAME_SIZE, 1), dtype=np.int)
      if (counter + batch_size >= len(dataset)):
        counter = 0
      for i in range(batch_size):
        input_frames[i] = np.expand_dims(dataset[counter+i:counter+i+N_SERIES], axis=3)
        output_frame[i] = np.expand_dims(dataset[counter+i+N_SERIES], axis=2)
      yield (input_frames, output_frame)
      counter += batch_size

In [ ]:
generator = DataGenerator(1, DATA_FILE_PATH, data_share=200)
x, y = next(generator)
print(f'X shape: {x.shape}.')
print(f'Y shape: {y.shape}.')

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

seq = keras.Sequential(
    [
        keras.Input(shape=(None, 100, 100, 1)),
        layers.ConvLSTM2D(
            filters=40, kernel_size=(3, 3), padding="same", return_sequences=True
        ),
        layers.BatchNormalization(),
        layers.ConvLSTM2D(
            filters=40, kernel_size=(3, 3), padding="same", return_sequences=True
        ),
        layers.BatchNormalization(),
        layers.ConvLSTM2D(
            filters=40, kernel_size=(3, 3), padding="same", return_sequences=True
        ),
        layers.BatchNormalization(),
        layers.ConvLSTM2D(
            filters=40, kernel_size=(3, 3), padding="same", return_sequences=True
        ),
        layers.BatchNormalization(),
        layers.Conv3D(
            filters=1, kernel_size=(3, 3, 3), activation="sigmoid", padding="same"
        ),
    ]
)
seq.compile(loss="binary_crossentropy", optimizer="adadelta",  metrics=['acc'])


In [ ]:
generator = DataGenerator(10, DATA_FILE_PATH, data_share=500)

seq.fit(generator)

In [ ]:
# generator = DataGenerator()

# for _ in range(2):
#   fig, axs = plt.subplots(1,2, figsize=(10, 10))
#   input, output = next(generator)
#   axs[0].imshow(input[-1][:,:,0])
#   axs[0].set_title('Input')
#   axs[1].imshow(output[0][:,:,0])
#   axs[1].set_title('Output')
#   plt.show()

In [ ]:
# es_callback = EarlyStopping('accuracy', min_delta=0.1, patience=1, restore_best_weights=True)

In [ ]:
# def model():
#   input = Input(shape=[100,100,1])
#   output = Conv2D(3, (3, 3), activation='relu', padding='same', input_shape=(100, 100, N_SERIES))(input)

#   model = Model(inputs=input, outputs=output)
#   model.compile(optimizer='Adam', loss='mse', metrics=['accuracy'])
#   return model

# generator = DataGenerator()
# model = model()
# h = model.fit(generator, epochs=EPOCHS)

In [ ]:
# generator = DataGenerator()
# x, y = next(generator) 
# predictions = model.predict()